# **Práctica 2**
* **Alumno 1**: Bolinches Segovia, Jorge
* **Alumno 2**: Cerezo Pomykol, Jan
***

### **Índice**
* [Carga de datasets](#1)
* [Entrenamiento y evaluación del Perceptrón](#2)
* [Ensemble del clasificador](#3)

### **Carga de datasets** <a class="anchor" id="1"></a>

In [1]:
import numpy as np
import pandas as pd
import os.path
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, fetch_openml

# Iris
dataset = load_iris()
X_iris = dataset.data
y_iris = dataset.target

# Wine
dataset = load_wine()
X_wine = dataset.data
y_wine = dataset.target

# Cancer
dataset = load_breast_cancer()
X_cancer = dataset.data
y_cancer = dataset.target

# Isolet
# Si existe la base de datos, cargo las variables
if os.path.exists("isolet_X.pickle"):
    X = pd.read_pickle('isolet_X.pickle')
    y = pd.read_pickle('isolet_y.pickle')
else:
    # Cargamos desde internet ( https://www.openml.org ) y la guardamos en el directorio local
    X, y = fetch_openml('isolet', version=1, return_X_y=True, cache=False)
    # Guardamos los datos para no volver a descargarlos
    X.to_pickle("isolet_X.pickle")
    y.to_pickle("isolet_y.pickle")

X_isolet = np.array(X)
y_isolet = pd.factorize(y)[0]

# MNIST
# Si existe la base de datos, cargo las variables
if os.path.exists("mnist_X.pickle"):
    X = pd.read_pickle('mnist_X.pickle')
    y = pd.read_pickle('mnist_y.pickle')
else:
    # Cargamos desde internet ( https://www.openml.org ) y la guardamos en el directorio local
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True, cache=False)
    # Guardamos los datos para no volver a descargarlos
    X.to_pickle("mnist_X.pickle")
    y.to_pickle("mnist_y.pickle")

X_mnist = np.array(X)
y_mnist = pd.factorize(y)[0]

datasets = {"iris": (X_iris, y_iris),
            "wine": (X_wine, y_wine),
            "cancer": (X_cancer, y_cancer),
            "isolet": (X_isolet, y_isolet),
            "mnist": (X_mnist, y_mnist)}

In [2]:
print("Dimensiones de los datasets:")
for i in datasets:
    print(i, ":\t", datasets[i][0].shape, sep='')

Dimensiones de los datasets:
iris:	(150, 4)
wine:	(178, 13)
cancer:	(569, 30)
isolet:	(7797, 617)
mnist:	(70000, 784)


## **Entrenamiento y evaluación del Perceptrón** <a class="anchor" id="2"></a>

In [3]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from time import time

Para la evaluación del perceptrón vamos a emplear la siguiente función, que recibe como parámetros el dataset (X, y) y la lista de valores de fracción de validación. Devuelve como resultado una lista con las tasas de acierto para cada valor de esta lista, así como el índice del mejor valor. Por cada valor de la lista de valores de fracción de validación interna, entrena y evalúa un perceptrón para cada dataset proporcionado por `KFold`. El ratio de aprendizaje $\delta$ toma el valor $\frac{1}{k}$, donde $k$ es el orden de la iteración. Este parámetro se corresponde con el atributo `eta0` de la clase Perceptron. En la primera iteración se le asigna valor 1, dado que el número de la iteración es accesible sólo después de ejecutar la primera.

In [4]:
def find_val(X, y, v):
    nv = len(v)
    scores = np.empty((nv, 10))
    scores_mean = np.empty(nv)
    best_v = 0
    prev = 0
    for vi in range(nv):
        clf = Perceptron(early_stopping=True, random_state=0, n_jobs=-1, validation_fraction=v[vi])
        kf = KFold(n_splits=10)
        n_iter = 1
        inicial = True
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            if inicial:
                clf.set_params(eta0=1)
                inicial = False
            else:
                clf.set_params(eta0=1/clf.n_iter_)
            scores[vi][n_iter-1] = clf.fit(X_train, y_train).score(X_test, y_test)
            n_iter = n_iter + 1
        scores_mean[vi] = np.mean(scores[vi])
        if scores_mean[vi] > prev:
            best_v = vi
            prev = scores_mean[vi]
    return scores_mean, best_v

## Dataset iris

In [5]:
t0 = time()
interval = 0.1
val = np.arange(0.02, 0.4, 0.005)
scores, best_v = find_val(X_iris, y_iris, val)
print("iris: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

iris: max_score: 0.6667, std: 0.0843, best_val: 0.0250, time: 6.2058s


## Dataset wine

In [6]:
t0 = time()
interval = 0.1
val = np.arange(0.1, 0.9, 0.005)
scores, best_v = find_val(X_wine, y_wine, val)
print("wine: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

wine: max_score: 0.8598, std: 0.1814, best_val: 0.4350, time: 14.2372s


## Dataset cancer

In [7]:
t0 = time()
interval = 0.1
val = np.arange(0.1, 0.9, 0.001)
scores, best_v = find_val(X_cancer, y_cancer, val)
print("cancer: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

cancer: max_score: 0.8947, std: 0.0666, best_val: 0.7330, time: 21.0799s


## Dataset isolet

In [8]:
t0 = time()
interval = 0.1
val = np.arange(0.1, 0.9, 0.01)
scores, best_v = find_val(X_isolet, y_isolet, val)
print("isolet: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

isolet: max_score: 0.9277, std: 0.0217, best_val: 0.1300, time: 444.1929s


## Dataset MNIST

In [9]:
t0 = time()
val = np.arange(0.05, 1, 0.05)
scores, best_v = find_val(X_mnist, y_mnist, val)
print("mnist: ", "max_score: %.4f" % np.max(scores), ", std: %.4f" % np.std(scores), ", best_val: %.4f" % val[best_v], ", time: %.4fs" % (time()-t0), sep='')

mnist: max_score: 0.8688, std: 0.0081, best_val: 0.7000, time: 533.4743s


***
#### **Resumen de resultados**

**Nota**: no hemos buscado mejores valores, dado que tarda demasiado tiempo en ejecutar la función con datasets grandes.

La siguiente tabla muestra la mejor tasa de aciertos que hemos conseguido con el perceptrón para cada dataset, así como el valor de la fracción de validación interna seleccionado.

|dataset|fraccion de validación interna|tasa de aciertos|
|---|---|---|
|iris|0.0250|0.6667|
|wine|0.4350|0.8598|
|cancer|0.7330|0.8947|
|isolet|0.1300|0.9277|
|mnist|0.7000|0.8688|

#### **Comentarios sobre los resultados**

Por lo general, los resultados no han sido tan buenos comparados con los resultados obtenidos en la práctica de la primera parte de la asignatura. Esto se debe a que los clasificadores que vimos tienen la capacidad de clasificar elementos en grupos linealmente no separables. En cambio, la función discriminante del perceptrón es lineal, por tanto hay datasets en los que no proporciona tan buenos resultados. Un ejemplo claro de esto es el dataset iris. Este dataset cuenta con tres clases, de cuales una está separada de las otras dos, pero estas dos últimas se entremezclan. Esto trae como consecuencia el hecho de que no se puede dividir linealmente el conjunto de datos (además de que no son clases separables). El perceptrón asigna correctamente los datos a la primera clase, pero las asignaciones de las otras dos son erróneas en la mitad de las veces. El mejor valor de fracción de validación interna que hemos encontrado para este dataset es 0.025, que da una tasa de aciertos del 66.67% (33.33% de fallos). En cuanto a los resultados obtenidos en los demás datasets, no son tan malos comparados con los de iris. Los mejores resultados se han obtenido con el dataset isolet, con una tasa de aciertos del 92.77% (7.2% de fallos).

## **Ensemble del clasificador** <a class="anchor" id="3"></a>

A continuación se incluye nuestra implementación necesaria para crear un ensemble de perceptrones, el cual usaremos para evaluar los 5 datasets vistos anteriormente. Además, comentaremos los resultados y los compararemos con otra función de sklearn que permite realizar ensembles.

### **Código de la función**


In [10]:
from sklearn.ensemble import VotingClassifier
def ensembler(X, y, name):
    X_aux = np.copy(X)
    y_aux = np.copy(y)
    estimators = []
    i = 0
    anterior = 0

    #mientras haya fallos o sea la primera iteración
    while(len(np.unique(y_aux))>1 or i==0):

        #se crea un perceptron
        clf = Perceptron(eta0=0.5, early_stopping=True, random_state=0)

        #se entrena con el dataset de errores de la iteración anterior
        clf = clf.fit(X_aux, y_aux)

        #se añade a la lista de clasificadores
        estimators.append((str(i), clf))

        #se crea el ensemble con la lista de clasificadores
        Vclf = VotingClassifier(estimators=estimators, voting = 'hard', n_jobs= -1)

        #se entrena el ensemble
        Vclf = Vclf.fit(X, y)

        #se comprueba los aciertos del ensemble
        predicts = Vclf.predict(X)
        scores = cross_val_score(Vclf, X, y, scoring='accuracy', cv=10)
        tasa = scores.mean()

        #Se extraen los datos mal clasificados por el ensemble y sus correspondientes etiquetas
        X_aux = X[y!=predicts]
        y_aux = y[y!=predicts]

        #Se comprueba que la tasa de acierto haya mejorado con respecto a la anterior iteración
        # que la tasa de acierto sea mayor que el 95%
        # y que el numero de clases en las que ha fallado sea mayor que 1 como condiciones de parada

        v, c = np.unique(y_aux, return_counts=True)
        if(tasa<=anterior or tasa>0.95 or np.sum(c<2)):
            break

        #se incrementa la iteración y se actualiza la tasa de acierto para la siguiente iteración
        i+=+1
        anterior = tasa
    print(name + ":\tscore: %0.4f (std: %0.4f)" % (tasa, scores.std()))

### **Evaluación de los datasets con la implementación**

In [11]:
t0 = time()
ensembler(X_iris, y_iris, 'iris')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_wine,y_wine, 'wine')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_cancer, y_cancer, 'cancer')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_isolet, y_isolet, 'isolet')
print("\ttime: %.4fs" % (time()-t0))
t0 = time()
ensembler(X_mnist, y_mnist, 'mnist')
print("\ttime: %.4fs" % (time()-t0))

iris:	score: 0.6667 (std: 0.0000)
	time: 3.3788s
wine:	score: 0.5297 (std: 0.1177)
	time: 0.4461s
cancer:	score: 0.7823 (std: 0.2063)
	time: 0.1140s
isolet:	score: 0.9259 (std: 0.0147)
	time: 23.7873s
mnist:	score: 0.8687 (std: 0.0144)
	time: 194.0456s


***
### **Resumen de resultados**


|dataset|Tasa de aciertos con el perceptrón|Tasa de aciertos con el ensemble|
|---|---|---|
|iris|0.6667|0.6667|
|wine|0.8598|0.5297|
|cancer|0.8947|0.7823|
|isolet|0.9277|0.9259|
|mnist|0.8688|0.8687|

Como podemos observar, los resultados son parecidos a los obtenidos con un sólo perceptrón, exceptuando el dataset wine.
Para el dataset Iris, el resultado es exactamente el mismo debido a que los fallos que comete el primer perceptrón pertenecen a una única clase, asi que no es posible entrenar varios perceptrones y por tanto el ensemble consta de uno sólo e igual al perceptrón de la parte anterior. Los resultados de wine son mucho peores, probablemente porque el dataset wine es muy dependiente de la fracción de validación y del ratio de apredizaje y en esta función no se ha implementado la modificación de los parámetros de los perceptrones del ensemble. La misma lógica se puede aplicar al dataset cáncer. Los resultados de isolet y mnist son bastante más parecidos a lo esperado


### **Evaluación de los dataset usando el ensemble de sklearn**

En esta parte, emplearemos el Adaptive Boosting de sklearn para crear ensembles con los que evaluar el dataset. Cabe destacar que una de las principales ventajas del Adaptive booster frente al nuestro es que este ensemble nos permite modificar el parámetro de la fracción de validación, por lo que se han usado los parámetros encontrados en la primera parte de la práctica y por tanto los más óptimos para cada dataset

Import necesario para usar el Adaptive booster de sklearn

In [12]:
from sklearn.ensemble import AdaBoostClassifier

#### Dataset iris

In [13]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.0250, n_jobs=-1)
t0 = time()

#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_iris, y_iris)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_iris, y_iris, scoring='accuracy', cv=10)

print("iris:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Iris'))

iris:	score: 0.8133, std: 0.1293, time: 0.4341s


#### Dataset wine

In [14]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.4350, n_jobs=-1)
t0 = time()

#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_wine, y_wine)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_wine, y_wine, scoring='accuracy', cv=10)

print("wine:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Wine'))

wine:	score: 0.6304, std: 0.0753, time: 0.4173s


#### Dataset cancer

In [15]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.7330, n_jobs=-1)
t0 = time()

#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_cancer, y_cancer)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_cancer, y_cancer, scoring='accuracy', cv=10)

print("cancer:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Cancer'))

cancer:	score: 0.8823, std: 0.0422, time: 0.1029s


#### Dataset isolet

In [16]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.1300, n_jobs=-1)
t0 = time()

#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_isolet, y_isolet)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_isolet, y_isolet, scoring='accuracy', cv=10)

print("isolet:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Isolet'))

isolet:	score: 0.9256, std: 0.0153, time: 32.5279s


#### Dataset mnist

In [17]:
#Crear el perceptron
clf = Perceptron(early_stopping=True, random_state=0, validation_fraction=0.7000, n_jobs=-1)
t0 = time()

#Crear el ensemble indicando que queremos hasta 5 perceptrones de los anteriores
Ada = AdaBoostClassifier(base_estimator=clf, n_estimators=5, learning_rate=0.5, algorithm='SAMME', random_state=0)

#Entrenar el ensemble
Ada = Ada.fit(X_mnist, y_mnist)

#evaluación con validación cruzada con k=10
scores = cross_val_score(Ada, X_mnist, y_mnist, scoring='accuracy', cv=10)

print("mnist:\tscore: %0.4f, std: %0.4f, time: %.4fs" % (scores.mean(), scores.std(), time()-t0))
#print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Mnist'))

mnist:	score: 0.8796, std: 0.0080, time: 147.6491s


***
### **Comparativa de resultados**

|dataset|Tasa de aciertos con el ensemble implementado|Tasa de aciertos con el ensemble de sklearn|
|---|---|---|
|iris|0.6667|0.8133|
|wine|0.5297|0.6304|
|cancer|0.7823|0.8823|
|isolet|0.9259|0.9259|
|mnist|0.8687|0.8796|

En cuanto a los resultados de la comparativa, cabe destacar que la técnica de Adaptive boosting es distinta a la vista en clase e implementada por nosotros, lo que permite evaluar con 5 perceptrones el dataset Iris, por lo que la precisión del AdaBoost es mucho mayor a la nuestra. Para los datasets de wine y cancer, el AdaBoost permite modificar la fracción de validación, por lo que obtiene mayor precisión que nuestro ensemble. En cuanto a los datasets isolet y mnist, los parámetros de los perceptrones no son tan influyentes y el tamaño de los datasets no limita nuestro algoritmo a la hora de crear perceptrones, es por eso por lo que los resultados obtenidos por ambos ensembles son muy similares, siendo AdaBoost solo mejor en mnist en un 1%.